In [1]:
from matplotlib import pyplot
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.imputation.mice as mice
import matplotlib.pyplot as plt
%matplotlib inline

## Load Data

In [2]:
train16 = pd.read_csv("train_2016_v2.csv", parse_dates=["transactiondate"])
print(train16.shape)
train16.tail()

(90275, 3)


,parcelid,logerror,transactiondate
90270,10774160,-0.0356,2016-12-30
90271,12046695,0.0070,2016-12-30
90272,12995401,-0.2679,2016-12-30
90273,11402105,0.0602,2016-12-30
90274,12566293,0.4207,2016-12-30


In [3]:
property16 = pd.read_csv("properties_2016.csv")
print(property16.shape)
property16.tail()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2985217, 58)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
2985212,168176230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,14273630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,168040630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,168040830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985216,168040430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
new_train16 = train16.set_index('parcelid').join(property16.set_index('parcelid'))
new_train16.reset_index(inplace=True, drop=True)
new_train16.head()

,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,0.0276,2016-08-02,1.0,NaN,NaN,3.0,4.0,NaN,4.0,3.0,...,NaN,NaN,245180.0,567112.0,2015.0,321932.0,7219.18,NaN,NaN,6.037113e+13
1,-0.0182,2016-08-02,1.0,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,NaN,NaN,254691.0,459844.0,2015.0,205153.0,6901.09,NaN,NaN,6.037113e+13
2,-0.1009,2016-05-03,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,...,NaN,NaN,235114.0,384787.0,2015.0,149673.0,4876.61,NaN,NaN,6.037113e+13
3,-0.0121,2016-04-05,1.0,NaN,NaN,2.0,4.0,NaN,4.0,2.0,...,NaN,NaN,262309.0,437176.0,2015.0,174867.0,5560.07,NaN,NaN,6.037113e+13
4,-0.0481,2016-07-15,1.0,NaN,NaN,2.0,4.0,NaN,4.0,2.0,...,NaN,NaN,232037.0,382055.0,2015.0,150018.0,4878.25,NaN,NaN,6.037113e+13


## Imputation

In [5]:
imp = mice.MICEData(new_train16[['landtaxvaluedollarcnt', \
                                 'taxvaluedollarcnt', 'taxamount', 'regionidzip', 'structuretaxvaluedollarcnt', \
                                 'censustractandblock', 'calculatedfinishedsquarefeet', 'yearbuilt', 'fullbathcnt', \
                                 'calculatedbathnbr', 'regionidcity', 'finishedsquarefeet12',"lotsizesquarefeet"\
                                ,"unitcnt", "buildingqualitytypeid", "heatingorsystemtypeid", "regionidneighborhood",\
                                "garagetotalsqft", "garagecarcnt"]])
imp

In [8]:
imp_df = imp.data
imp_df.head()

,landtaxvaluedollarcnt,taxvaluedollarcnt,taxamount,regionidzip,structuretaxvaluedollarcnt,censustractandblock,calculatedfinishedsquarefeet,yearbuilt,fullbathcnt,calculatedbathnbr,regionidcity,finishedsquarefeet12,lotsizesquarefeet,unitcnt,buildingqualitytypeid,heatingorsystemtypeid,regionidneighborhood,garagetotalsqft,garagecarcnt
0,321932.0,567112.0,7219.18,96339.0,245180.0,6.037113e+13,2538.0,1978.0,3.0,3.0,12447.0,2538.0,11012.0,1.0,4.0,2.0,268588.0,0.478772,0.187995
1,205153.0,459844.0,6901.09,96339.0,254691.0,6.037113e+13,1589.0,1959.0,3.0,3.0,12447.0,1589.0,11010.0,1.0,4.0,2.0,268588.0,0.478772,0.187995
2,149673.0,384787.0,4876.61,96339.0,235114.0,6.037113e+13,2411.0,1973.0,2.0,2.0,12447.0,2411.0,11723.0,1.0,4.0,2.0,268588.0,0.478772,0.187995
3,174867.0,437176.0,5560.07,96339.0,262309.0,6.037113e+13,2232.0,1973.0,2.0,2.0,12447.0,2232.0,9002.0,1.0,4.0,2.0,268588.0,0.478772,0.187995
4,150018.0,382055.0,4878.25,96339.0,232037.0,6.037113e+13,1882.0,1973.0,2.0,2.0,12447.0,1882.0,9002.0,1.0,4.0,2.0,268588.0,0.478772,0.187995


In [9]:
new_train16["landtaxvaluedollarcnt"] = imp_df["landtaxvaluedollarcnt"]
new_train16["taxvaluedollarcnt"] = imp_df["taxvaluedollarcnt"]
new_train16["taxamount"] = imp_df["taxamount"]
new_train16["regionidzip"] = imp_df["regionidzip"]
new_train16["structuretaxvaluedollarcnt"] = imp_df["structuretaxvaluedollarcnt"]
new_train16["censustractandblock"] = imp_df["censustractandblock"]
new_train16["calculatedfinishedsquarefeet"] = imp_df["calculatedfinishedsquarefeet"]
new_train16["yearbuilt"] = imp_df["yearbuilt"]
new_train16["fullbathcnt"] = imp_df["fullbathcnt"]
new_train16["calculatedbathnbr"] = imp_df["calculatedbathnbr"]
new_train16["regionidcity"] = imp_df["regionidcity"]
new_train16["finishedsquarefeet12"] = imp_df["finishedsquarefeet12"]
new_train16["lotsizesquarefeet"] = imp_df["lotsizesquarefeet"]
new_train16["unitcnt"] = imp_df["unitcnt"]
new_train16["buildingqualitytypeid"] = imp_df["buildingqualitytypeid"]
new_train16["heatingorsystemtypeid"] = imp_df["heatingorsystemtypeid"]
new_train16["regionidneighborhood"] = imp_df["regionidneighborhood"]
new_train16["garagetotalsqft"] = imp_df["garagetotalsqft"]
new_train16["garagecarcnt"] = imp_df["garagecarcnt"]

## OLS

In [10]:
model = sm.OLS.from_formula("logerror ~ scale(latitude) + scale(longitude) + C(propertylandusetypeid)\
                             + scale(rawcensustractandblock) + C(regionidcity) + C(regionidcounty)\
                             + scale(fullbathcnt) + C(regionidzip) + C(yearbuilt) + scale(structuretaxvaluedollarcnt)\
                             + scale(taxvaluedollarcnt) + C(assessmentyear) + scale(landtaxvaluedollarcnt)\
                             + scale(taxamount) + scale(roomcnt) + C(fips) + scale(censustractandblock)\
                             + scale(calculatedbathnbr) + scale(calculatedfinishedsquarefeet) + scale(bathroomcnt)\
                             + scale(finishedsquarefeet12) + scale(bedroomcnt) + scale(lotsizesquarefeet)\
                             + scale(unitcnt) + C(buildingqualitytypeid) + C(heatingorsystemtypeid)\
                             + scale(regionidneighborhood) + scale(garagetotalsqft) + scale(garagecarcnt)", data=new_train16)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     2.876
Date:                Thu, 05 Jul 2018   Prob (F-statistic):          6.49e-133
Time:                        16:09:29   Log-Likelihood:                 37803.
No. Observations:               90275   AIC:                        -7.411e+04
Df Residuals:                   89529   BIC:                        -6.709e+04
Df Model:                         745                                         
Covariance Type:            nonrobust                                         
                                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

## Predict

In [11]:
train = new_train16[["latitude", "longitude", "propertylandusetypeid",\
                             "rawcensustractandblock", "regionidcity", "regionidcounty",\
                             "fullbathcnt", "regionidzip", "yearbuilt", "structuretaxvaluedollarcnt",\
                             "taxvaluedollarcnt", "assessmentyear", "landtaxvaluedollarcnt",\
                             "taxamount", "roomcnt", "fips", "censustractandblock",\
                             "calculatedbathnbr", "calculatedfinishedsquarefeet", "bathroomcnt",\
                             "finishedsquarefeet12", "bedroomcnt", "lotsizesquarefeet",\
                             "unitcnt", "buildingqualitytypeid", "heatingorsystemtypeid",\
                             "regionidneighborhood", "garagetotalsqft", "garagecarcnt"]]
#train = new_train16[col]

In [16]:
pred = result.predict(train)
pred[:10]

0    0.007698
1    0.003451
2    0.010885
3    0.008739
4    0.004387
5    0.004605
6    0.006577
7    0.023017
8    0.028112
9    0.008741
dtype: float64

## Score

In [13]:
def MAE(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

print(MAE(new_train16["logerror"], pred))

0.06937453916125083
